In [10]:
import os
from spacy.en import English
import numpy as np
import string
from keras.preprocessing.sequence import pad_sequences

In [11]:
# config
pan15train = "/data/pan15-authorship-verification-training-dataset-english-2015-04-19/"
pan15test = "/data/pan15-authorship-verification-test-dataset2-english-2015-04-19/"
pan14train = "/data/pan14-author-verification-training-corpus-english-essays-2014-04-22/"
pan14test = "/data/pan14-author-verification-test-corpus2-english-essays-2014-04-22/"
pan14train = "/data/pan14-author-verification-training-corpus-english-novels-2014-04-22/"
pan14test = "/data/pan14-author-verification-test-corpus2-english-novels-2014-04-22/"
char_embeddings_file = "glove.840B.300d-char.txt"  # full path to the character embeddings file

In [12]:
# load spacy, this takes a while
nlp_word = English(parse=False, tag=False, entity=False)
nlp_char = English(parse=False, tag=False, entity=False)
with open(char_embeddings_file) as f:
    nlp_char.vocab.load_vectors(f)

In [13]:
def read_file(filepath):
    with open(filepath) as f:
        s = f.read()
    return s

def load_pan_data(directory, prefix="E"):
    """Load known and unknown texts in the PAN data format"""
    # FIXME: assumes one known file per author, which is fine for English datasets only
    authors = sorted([x for x in os.listdir(directory) if x.startswith(prefix)])
    known_texts = []
    unknown_texts = []
    for author in authors:
        kf = os.path.join(directory, author, "known01.txt")
        uf = os.path.join(directory, author, "unknown.txt")
        known_texts.append(read_file(kf))
        unknown_texts.append(read_file(uf))
        
    truthfile = os.path.join(directory, "truth.txt")
    with open(truthfile) as f:
        lines = f.read().strip().split("\n")
    y = [1 if line.split()[1] == "Y" else 0 for line in lines]
    y = np.array(y)
    return known_texts, unknown_texts, y

def create_pairs(knownX, unknownX):
    print(len(knownX), len(unknownX))
    """Creates pairs of known and unknown texts"""
    pairs = []
    for i in range(len(knownX)):
        pairs += [[knownX[i], unknownX[i]]]
    pairs = np.array(pairs)
    print(pairs.shape)
    return pairs

def vectorize(text, nlp, target_sentences=60):
    """Convert text (string) to embeddings (numpy array)
    nlp should be an initialised Spacy pipeline with loaded embeddings"""
    p = nlp(text, entity=False, tag=False)
    vecs = [sent.vector for sent in p.sents]
    if len(vecs) > target_sentences:
        return vecs[:target_sentences]
    vecs += [nlp.vocab["<pad>"].vector] * (target_sentences - len(vecs))
    return np.array(vecs)   

def build_embedding_matrix(embedding_path, char_indices):    
    embeddings_index = {}
    f = open(embedding_path)
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    embedding_matrix = np.zeros((len(chars), 300))
    for char, i in char_indices.items():
        embedding_vector = embeddings_index.get(char)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def char_vectorize(text, char_indices, target_length=1000):
    text = text.lower()
    inds = []
    for char in text:
        if char in char_indices:
            inds.append(char_indices[char])
        else:
            inds.append(char_indices["+"])
    return inds

def char_vectorize_all(texts, char_indices, target_length=1000):
    vecs = [char_vectorize(t, char_indices) for t in texts]
    return pad_sequences(vecs, target_length)

punctuation = '''!"'#$%&'()*,-./:;<>?@[]_'''
# "+ for padding
chars = ["+"] + sorted(list(set(string.ascii_lowercase + punctuation + " ")))
char_indices = dict((c, i) for i, c in enumerate(chars))i
indices_char = dict((i, c) for i, c in enumerate(chars))

embedding_matrix = build_embedding_matrix('glove.840B.300d-char.txt', char_indices)

char_vectorize_all(["teset", "asdf"], char_indices)

array([[ 0,  0,  0, ..., 43, 29, 44],
       [ 0,  0,  0, ..., 43, 28, 30]], dtype=int32)

In [14]:
from __future__ import absolute_import
from __future__ import print_function
np.random.seed(1337)  # for reproducibility
import random

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Lambda, LSTM, Masking, Dropout, Embedding
from keras.optimizers import RMSprop, Adam
from keras import backend as K

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def cosine_distance(vects):
    x, y = vects
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    num_chars = 51
    embed_dim = 300
    max_seq_len = 1000
    
    
    embedding_layer = Embedding(num_chars,
                            embed_dim,
                            weights=[embedding_matrix],
                            input_length=max_seq_len,
                            trainable=True)
    sequence_input = Input(shape=(max_seq_len,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    seq = Sequential()
    seq.add(embedding_layer)
    seq.add(LSTM(128, activation='relu'))
    seq.add(Dense(128, activation='relu'))
    return seq

def compute_accuracy(predictions, labels):
    return np.mean(np.equal(predictions.ravel() < 0.5, labels))

def train(tr_pairs, tr_y):
    input_dim = (tr_pairs.shape[-1])

    # network definition
    base_network = create_base_network(input_dim,)

    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))

    # because we re-use the same instance `base_network`,
    # the weights of the network
    # will be shared across the two branches
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    model = Model(inputs=[input_a, input_b], outputs=distance)
    
    rms = Adam(lr=0.001, decay=1e-4)
    model.compile(loss=contrastive_loss, optimizer=rms)
    model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
              validation_split=0.015,
              batch_size=20,
              epochs=15)
    return model
       
def evaluate(model, tr_pairs, tr_y, te_pairs, te_y):
    # compute final accuracy on training and test sets
    pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
    tr_acc = compute_accuracy(pred, tr_y)
    pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
    te_acc = compute_accuracy(pred, te_y)
    print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
    print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))
    
def combine_vectors(v1s, v2s):
    return [np.hstack([v1s[i], v2s[i]]) for i in range(len(v1s))]

def run_experiment(known_train, unknown_train, known_test, unknown_test, y_train, y_test):
    tr_pairs = create_pairs(known_train, unknown_train)
    te_pairs = create_pairs(known_test, unknown_test)
    
    model = train(tr_pairs, y_train)
    evaluate(model, tr_pairs, y_train, te_pairs, y_test)

In [80]:
model = create_base_network(1000)

In [15]:
known_train, unknown_train, y_train = load_pan_data(pan15train)
known_test, unknown_test, y_test = load_pan_data(pan15test)

In [16]:
# word vectors
known_train_vecs = char_vectorize_all(known_train, char_indices)
known_test_vecs = char_vectorize_all(known_test, char_indices)
unknown_train_vecs = char_vectorize_all(unknown_train, char_indices)
unknown_test_vecs = char_vectorize_all(unknown_test, char_indices)

13

In [81]:
from statistics import mean
min([x.count(".") for x in known_train])

40

In [17]:
run_experiment(known_train_vecs, unknown_train_vecs, known_test_vecs, unknown_test_vecs, y_train, y_test)

100 100
(100, 2, 1000)
500 500
(500, 2, 1000)
Train on 98 samples, validate on 2 samples
Epoch 1/15
98/98 [==============================] - 25s - loss: 0.3679 - val_loss: 0.3620
Epoch 2/15
98/98 [==============================] - 22s - loss: 0.3029 - val_loss: 0.2142
Epoch 3/15
98/98 [==============================] - 20s - loss: 0.2954 - val_loss: 0.2167

In [46]:
tr_pairs[:, 0].shape

(100, 60, 300)

In [48]:
model = train(tr_pairs, y_train)

Train on 95 samples, validate on 5 samples
Epoch 1/5
95/95 [==============================] - 2s - loss: 4.6267 - val_loss: 0.4666
Epoch 2/5
95/95 [==============================] - 1s - loss: 0.3668 - val_loss: 0.4346
Epoch 3/5
95/95 [==============================] - 1s - loss: 0.3595 - val_loss: 0.4200

In [9]:
run_experiment(w_known_train_vecs, w_unknown_train_vecs, w_known_test_vecs, w_unknown_test_vecs, y_train, y_test)
# run_experiment(c_known_train_vecs, c_unknown_train_vecs, c_known_test_vecs, c_unknown_test_vecs, y_train, y_test)
# run_experiment(wc_known_train_vecs, wc_unknown_train_vecs, wc_known_test_vecs, wc_unknown_test_vecs, y_train, y_test)

NameError: name 'w_known_train_vecs' is not defined

In [63]:
knownXs = char_vectorize_all(["this is some text dsf asdf asdf ", "and some moer ateasdf asdf ", "we have three samples"], char_indices)
unknownXs = char_vectorize_all(["this is some text dsf asdf asdf ", "and some moer ateasdf asdf ", "we have three samples"], char_indices)

In [69]:
tr_pairs = create_pairs(knownXs, unknownXs)

3 3
(3, 2, 1000)


In [81]:
train(tr_pairs, [0,0,1])

Train on 2 samples, validate on 1 samples
Epoch 1/30
2/2 [==============================] - 3s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 2/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 3/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 4/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 5/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 6/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 7/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 8/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 9/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 10/30
2/2 [==============================] - 2s - loss: 1.0000 - val_loss: 0.0000e+00
Epoch 11/30
2/2 [==============================

In [2]:
with open("function_words.txt") as f:
    s = f.read().split("\n")
print(' '.join(s))

a about above after all although am among an and another any anybody anyone anything are around as at be because before behind below beside between both but by can cos do down each either enough every everybody everyone everything few following for from have he her him i if in including inside into is it its latter less like little lots many me more most much must my near need neither no nobody none nor nothing of off on once one onto opposite or our outside over own past per plenty plus regarding same several she should since so some somebody someone something such than that the their them these they this those though through till to toward towards under unless unlike until up upon us used via we what whatever when where whether which while who whoever whom whose will with within without worth would yes you 
